In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime 
import sys
%matplotlib inline
evaluation=pd.read_csv('submit.txt')
product_info=pd.read_csv('product_info.txt')
product_quantity=pd.read_csv('product_quantity.txt')
#Mytrain_day=pd.read_csv('Mytrain_day.csv')

In [ ]:
product_quantity=product_quantity.sort_values(['product_id','product_date'])
product_quantity['product_month']=product_quantity['product_date'].apply(lambda x: x[:7])
train_month=product_quantity.groupby(['product_id','product_month']).sum()['ciiquantity'].unstack()

In [ ]:
train_month

In [ ]:
def _map(data):
    empty = pd.DataFrame()
    for index, row in data.iterrows():   # 获取每行的index、row
        for col_name in data.columns:
            empty=empty.append({"product_id":row.name,"product_month":col_name,"ciiquantity":row[col_name]},ignore_index=True)            
    return empty

#train_month_t=_map(train_month)
#train_month_t.to_csv('train_month_t.csv',index=False)
train_month_t = pd.read_csv("train_month_t.csv")
train_month_t

In [ ]:
train_month_t[train_month_t.product_id==1].plot(x='product_month', y='ciiquantity',figsize=(15,6))
train_month_t[train_month_t.product_id==2].plot(x='product_month', y='ciiquantity',figsize=(15,6))

In [ ]:
#train_month_t = train_month_t.reset_index(drop=True)
train_month_t = train_month_t.dropna(axis=0,how='any')
train_month_t = train_month_t.reset_index(drop=True)
train_month_t

In [ ]:
import calendar
def get_week(product_month):
    _yyyy=int(product_month[0:4])    
    _mm=int(product_month[5:7])
    _first,_days=calendar.monthrange(_yyyy,_mm)
    arr = np.zeros(7)
    for i in range(_days):
        _week=(i+_first)%7
        arr[_week]+=1
    return arr
get_week('2014-01')


In [ ]:
def addweek(table):
    empty = pd.DataFrame()
    for index, row in table.iterrows():
        array=get_week(row['product_month'])        
        empty=empty.append({"w0":array[0],"w1":array[1],"w2":array[2],"w3":array[3],"w4":array[4],"w5":array[5],"w6":array[6]},ignore_index=True)
        sys.stdout.write('\r{}'.format(index + 1))
    table=pd.concat([table, empty], axis=1)
    return table

train_month_t_week=addweek(train_month_t)

In [ ]:
train_month_t_week['month']=train_month_t_week['product_month'].apply(lambda x:x[5:7])
dummies = pd.get_dummies(train_month_t_week['month'], prefix='month', drop_first=False)
train_month_t_week = pd.concat([train_month_t_week, dummies], axis=1)
train_month_t_week.drop('month', axis=1)
train_month_t_week

In [ ]:
def get_product_info(product_id):
    features=['eval','eval2','eval3','eval4','voters','maxstock']
    _i=0
    arr = np.zeros(6)
    for feature in features:
        mean, std = product_info[feature].mean(), product_info[feature].std()
        value=(product_info[product_info.product_id==product_id][feature] - mean)/std
        arr[_i]=value
        _i+= 1
    return arr
get_product_info(1)

In [ ]:
def addproduct_info(table):
    empty = pd.DataFrame()
    for index, row in table.iterrows():
        array=get_product_info(row['product_id'])        
        empty=empty.append({"f0":array[0],"f1":array[1],"f2":array[2],"f3":array[3],"f4":array[4],"f5":array[5]},ignore_index=True)
        sys.stdout.write('\r{}'.format(index + 1))
    table=pd.concat([table, empty], axis=1)
    return table

train_month_t_week_product_info=addproduct_info(train_month_t_week)

# 读取已处理数据

In [25]:
#train_month_t_week_product_info.to_csv('train_month_t_week_product_info.csv',index=False)
read_train_month_t_week_product_info = pd.read_csv("train_month_t_week_product_info.csv")

In [26]:
#['eval','eval2','eval3','eval4','voters','maxstock']
#['f0',' f1','f2','f3','f4','f5']
train_month_t_week_product_info = read_train_month_t_week_product_info
train_month_t_week_product_info["year"]=train_month_t_week_product_info["product_month"].apply(lambda x:x[0:4])
train_month_t_week_product_info["year_s"]=train_month_t_week_product_info["year"].apply(lambda x:(float(x)-2015.5)/4)
train_month_t_week_product_info["w0"]=train_month_t_week_product_info["w0"].apply(lambda x:x/10)
train_month_t_week_product_info["w1"]=train_month_t_week_product_info["w1"].apply(lambda x:x/10)
train_month_t_week_product_info["w2"]=train_month_t_week_product_info["w2"].apply(lambda x:x/10)
train_month_t_week_product_info["w3"]=train_month_t_week_product_info["w3"].apply(lambda x:x/10)
train_month_t_week_product_info["w4"]=train_month_t_week_product_info["w4"].apply(lambda x:x/10)
train_month_t_week_product_info["w5"]=train_month_t_week_product_info["w5"].apply(lambda x:x/10)
train_month_t_week_product_info["w6"]=train_month_t_week_product_info["w6"].apply(lambda x:x/10)
train_month_t_week_product_info=train_month_t_week_product_info.drop(['year', 'month', 'product_id', 'product_month', 'f0', 'f1', 'f2'],axis=1)

In [27]:
print(train_month_t_week_product_info.columns)
train_month_t_week_product_info

Index(['ciiquantity', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'month_01',
       'month_02', 'month_03', 'month_04', 'month_05', 'month_06', 'month_07',
       'month_08', 'month_09', 'month_10', 'month_11', 'month_12', 'f3', 'f4',
       'f5', 'year_s'],
      dtype='object')


,ciiquantity,w0,w1,w2,w3,w4,w5,w6,month_01,month_02,...,month_07,month_08,month_09,month_10,month_11,month_12,f3,f4,f5,year_s
0,29.0,0.4,0.4,0.5,0.5,0.5,0.4,0.4,1,0,...,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
1,111.0,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0,1,...,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
2,13.0,0.5,0.4,0.4,0.4,0.4,0.5,0.5,0,0,...,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
3,71.0,0.4,0.5,0.5,0.4,0.4,0.4,0.4,0,0,...,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
4,74.0,0.4,0.4,0.4,0.5,0.5,0.5,0.4,0,0,...,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
5,30.0,0.5,0.4,0.4,0.4,0.4,0.4,0.5,0,0,...,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
6,55.0,0.4,0.5,0.5,0.5,0.4,0.4,0.4,0,0,...,1,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
7,159.0,0.4,0.4,0.4,0.4,0.5,0.5,0.5,0,0,...,0,1,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
8,35.0,0.5,0.5,0.4,0.4,0.4,0.4,0.4,0,0,...,0,0,1,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
9,134.0,0.4,0.4,0.5,0.5,0.5,0.4,0.4,0,0,...,0,0,0,1,0,0,-0.830884,-0.266098,-0.529165,-0.375


In [49]:
def random_read_train_data():    
    _features = train_month_t_week_product_info.drop('ciiquantity',axis=1).as_matrix()
    _targets = train_month_t_week_product_info['ciiquantity'].as_matrix()
    _targets.shape = (_targets.shape[0], 1)
    _targets.transpose()
    return _features,_targets
tf_features,tf_targets = random_read_train_data()
print(tf_features.shape[1])
tf_features[0:2],tf_targets[0:2]

23


(array([[ 0.4       ,  0.4       ,  0.5       ,  0.5       ,  0.5       ,
          0.4       ,  0.4       ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.83088377,
         -0.2660979 , -0.52916452, -0.375     ],
        [ 0.4       ,  0.4       ,  0.4       ,  0.4       ,  0.4       ,
          0.4       ,  0.4       ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.83088377,
         -0.2660979 , -0.52916452, -0.375     ]]), array([[  29.],
        [ 111.]]))

In [50]:
def add_layer(inputs, in_size, out_size, activation_function=None):    
    Weights = tf.Variable(tf.random_normal([in_size, out_size],mean=0.0,stddev=0.05))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
in_shape=tf_features.shape[1]
hidden_shape=20
l_r=0.01
x = tf.placeholder(tf.float32, shape=[None, in_shape])
y = tf.placeholder(tf.float32, [None,1])
l1 = add_layer(x, in_shape, hidden_shape, activation_function=tf.nn.sigmoid)
prediction = add_layer(l1, hidden_shape, 1, activation_function=None)

loss = tf.sqrt(tf.reduce_mean(tf.reduce_sum(tf.square(y - prediction),
                     reduction_indices=[1])))
train_step = tf.train.GradientDescentOptimizer(l_r).minimize(loss)
init_op = tf.initialize_all_variables()
saver = tf.train.Saver()
sess=tf.Session()
sess.run(init_op)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [51]:
#saver.restore(sess, "./model14.ckpt")
#saver.restore(sess, "./model_{}_{}.ckpt".format(tf_features.shape[1],hidden_shape)

In [58]:
print('Loss: {:.6f}'.format(sess.run(loss, feed_dict={x: tf_features, y: tf_targets})))
epochs=3000
start = datetime.now()
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
for epoch in range(epochs):    
    sess.run(train_step, feed_dict={x: tf_features, y: tf_targets})
    if epoch % 100 == 0:
        temploss = sess.run(loss, feed_dict={x: tf_features, y: tf_targets})
        end = datetime.now()
        sys.stdout.write('\rEpoch {:>2} Loss: {:.6f} time: {}'.format(epoch + 1,temploss,end-start))
        start = datetime.now()

Loss: 195.458817
Epoch 2901 Loss: 195.358658 time: 0:00:03.001580

In [60]:
save_path = saver.save(sess, "./model_{}_{}.ckpt".format(tf_features.shape[1],hidden_shape))
print(save_path)

./model_23_20.ckpt


In [59]:
tf_features,tf_targets = random_read_train_data()
ansow=sess.run(prediction, feed_dict={x: tf_features})
ansow

array([[  48.48693848],
       [  55.66024017],
       [  51.65536118],
       ..., 
       [ 125.58386993],
       [ 148.87397766],
       [  99.21736908]], dtype=float32)

In [ ]:
sess.close()

In [ ]:
'''神经网络测试
def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
_inputs=10
x_data = np.linspace(-1,1,10)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise
x_data = np.random.random((10,_inputs))
xs = tf.placeholder(tf.float32, [None, _inputs])
ys = tf.placeholder(tf.float32, [None, 1])
l1 = add_layer(xs, _inputs, 10, activation_function=tf.nn.relu)
prediction = add_layer(l1, 10, 1, activation_function=None)
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                     reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    # training train_step 和 loss 都是由 placeholder 定义的运算，所以这里要用 feed 传入参数
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        # to see the step improvement
        print(sess.run(loss, feed_dict={xs: x_data, ys: y_data}))
ansow=sess.run(prediction, feed_dict={xs: x_data})
ansow
#'''

# 被预测数据预处理

In [66]:
p_evaluation=evaluation
p_evaluation

,product_id,product_month,ciiquantity_month
0,1,2015-12-01,100
1,2,2015-12-01,100
2,3,2015-12-01,100
3,4,2015-12-01,100
4,5,2015-12-01,100
5,6,2015-12-01,100
6,7,2015-12-01,100
7,8,2015-12-01,100
8,9,2015-12-01,100
9,10,2015-12-01,100


In [67]:
p_evaluation['product_month']=p_evaluation['product_month'].apply(lambda x: x[:7])
p_evaluation

,product_id,product_month,ciiquantity_month
0,1,2015-12,100
1,2,2015-12,100
2,3,2015-12,100
3,4,2015-12,100
4,5,2015-12,100
5,6,2015-12,100
6,7,2015-12,100
7,8,2015-12,100
8,9,2015-12,100
9,10,2015-12,100


In [69]:
import calendar
def get_week(product_month):
    _yyyy=int(product_month[0:4])    
    _mm=int(product_month[5:7])
    _first,_days=calendar.monthrange(_yyyy,_mm)
    arr = np.zeros(7)
    for i in range(_days):
        _week=(i+_first)%7
        arr[_week]+=1
    return arr
get_week('2014-01')

array([ 4.,  4.,  5.,  5.,  5.,  4.,  4.])

In [70]:
def addweek(table):
    empty = pd.DataFrame()
    for index, row in table.iterrows():
        array=get_week(row['product_month'])        
        empty=empty.append({"w0":array[0],"w1":array[1],"w2":array[2],"w3":array[3],"w4":array[4],"w5":array[5],"w6":array[6]},ignore_index=True)
        sys.stdout.write('\r{}'.format(index + 1))
    table=pd.concat([table, empty], axis=1)
    return table

p_evaluation=addweek(p_evaluation)
p_evaluation

56000

,product_id,product_month,ciiquantity_month,w0,w1,w2,w3,w4,w5,w6
0,1,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
1,2,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
2,3,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
3,4,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
4,5,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
5,6,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
6,7,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
7,8,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
8,9,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0
9,10,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0


In [72]:
def get_product_info(product_id):
    features=['eval','eval2','eval3','eval4','voters','maxstock']
    _i=0
    arr = np.zeros(6)
    for feature in features:
        mean, std = product_info[feature].mean(), product_info[feature].std()
        value=(product_info[product_info.product_id==product_id][feature] - mean)/std
        arr[_i]=value
        _i+= 1
    return arr
get_product_info(1)

array([-0.80049081,  0.26134466, -0.71201506, -0.83088377, -0.2660979 ,
       -0.52916452])

In [73]:
def addproduct_info(table):
    empty = pd.DataFrame()
    for index, row in table.iterrows():
        array=get_product_info(row['product_id'])        
        empty=empty.append({"f0":array[0],"f1":array[1],"f2":array[2],"f3":array[3],"f4":array[4],"f5":array[5]},ignore_index=True)
        sys.stdout.write('\r{}'.format(index + 1))
    table=pd.concat([table, empty], axis=1)
    return table

p_evaluation=addproduct_info(p_evaluation)

56000

,product_id,product_month,ciiquantity_month,w0,w1,w2,w3,w4,w5,w6,f0,f1,f2,f3,f4,f5
0,1,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
1,2,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,-0.442614,-0.712015,-0.111140,0.063709,0.020579
2,3,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,-0.442614,-0.320852,0.368689,0.079391,0.111258
3,4,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,-0.442614,-0.320852,0.128774,-0.250906,-0.296799
4,5,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,0.261345,-0.712015,-0.111140,0.309717,-0.415815
5,6,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,1.669262,-0.712015,0.128774,0.103404,-0.104105
6,7,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,1.669262,-0.712015,0.368689,-0.051454,-0.217455
7,8,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,0.965303,-0.712015,-0.351055,0.107324,-0.455488
8,9,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,1.669262,-0.712015,-0.111140,-0.133783,-0.228789
9,10,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,-0.800491,-0.442614,-0.712015,-0.111140,0.381755,0.235942


In [76]:
p_evaluation['month']=p_evaluation['product_month'].apply(lambda x:x[5:7])
dummies = pd.get_dummies(p_evaluation['month'], prefix='month', drop_first=False)
p_evaluation = pd.concat([p_evaluation, dummies], axis=1)
p_evaluation.drop('month', axis=1)
p_evaluation

,product_id,product_month,ciiquantity_month,w0,w1,w2,w3,w4,w5,w6,...,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,1,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
1,2,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
2,3,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
3,4,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
4,5,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
5,6,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
6,7,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
7,8,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
8,9,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1
9,10,2015-12,100,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,1


In [80]:
#p_evaluation.to_csv('p_evaluation.csv',index=False)
p_evaluation = pd.read_csv("p_evaluation.csv")
p_evaluation.columns

Index(['product_id', 'product_month', 'ciiquantity_month', 'w0', 'w1', 'w2',
       'w3', 'w4', 'w5', 'w6', 'f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'month',
       'month_01', 'month_02', 'month_03', 'month_04', 'month_05', 'month_06',
       'month_07', 'month_08', 'month_09', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [79]:
read_train_month_t_week_product_info = pd.read_csv("train_month_t_week_product_info.csv")
read_train_month_t_week_product_info.columns

Index(['ciiquantity', 'product_id', 'product_month', 'w0', 'w1', 'w2', 'w3',
       'w4', 'w5', 'w6', 'month', 'month_01', 'month_02', 'month_03',
       'month_04', 'month_05', 'month_06', 'month_07', 'month_08', 'month_09',
       'month_10', 'month_11', 'month_12', 'f0', 'f1', 'f2', 'f3', 'f4', 'f5'],
      dtype='object')